In this notebook, we will learn XGBoost, and then, we'll create a model for a telecommunication company, to predict when its customers will leave for a competitor, so that they can take some action to retain the customers. I have earlier published a solution using logistic regresion if you want to come the XGBoost performance with Logistic Regression checkout the notebook for more details <a href="https://www.kaggle.com/anmol4210/customerchurnpredictionusinglogisticregression">Logistic Regression Notebook</a>


# Introduction

<a id="ref1"></a>

## What is XGBoost?

XGBoost is a decision-tree-based ensemble Machine Learning algorithm that uses a gradient boosting framework. In prediction problems involving unstructured data (images, text, etc.) artificial neural networks tend to outperform all other algorithms or frameworks. However, when it comes to small-to-medium structured/tabular data, decision tree based algorithms are considered best-in-class right now. Please see the chart below for the evolution of tree-based algorithms over the years.
<img src="https://miro.medium.com/max/875/1*QJZ6W-Pck_W7RlIDwUIN9Q.jpeg" width="600" align="center">
## Why does XGBoost perform so well?
XGBoost and Gradient Boosting Machines (GBMs) are both ensemble tree methods that apply the principle of boosting weak learners (CARTs generally) using the gradient descent architecture. However, XGBoost improves upon the base GBM framework through systems optimization and algorithmic enhancements.

<img src="https://miro.medium.com/max/875/1*FLshv-wVDfu-i54OqvZdHg.png" width="600" align="center">

### Customer churn with XGBoost

A telecommunications company is concerned about the number of customers leaving their land-line business for cable competitors. They need to understand who is leaving. Imagine that you are an analyst at this company and you have to find out who is leaving and why.


Lets first import required libraries:


In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


# <h2 id="about_dataset">About the dataset</h2>
We will use a telecommunications dataset for predicting customer churn. This is a historical customer dataset where each row represents one customer. The data is relatively easy to understand, and you may uncover insights you can use immediately. Typically it is less expensive to keep customers than acquire new ones, so the focus of this analysis is to predict the customers who will stay with the company. 

This data set provides information to help you predict what behavior will help you to retain customers. You can analyze all relevant customer data and develop focused customer retention programs.

The dataset includes information about:

-   Customers who left within the last month – the column is called Churn
-   Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
-   Customer account information – how long they had been a customer, contract, payment method, paperless billing, monthly charges, and total charges
-   Demographic info about customers – gender, age range, and if they have partners and dependents


### Load the Telco Churn data

Telco Churn is a hypothetical data file that concerns a telecommunications company's efforts to reduce turnover in its customer base. Each case corresponds to a separate customer and it records various demographic and service usage information.


In [ ]:
churn_df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
churn_df.head()

<h2 id="preprocessing">Data pre-processing and selection</h2>


Lets select some features for the modeling. Also we change the target data type to be integer, as it is a requirement  by the skitlearn algorithm:


In [ ]:
churn_df = churn_df[['tenure','Dependents', 'MultipleLines', 'InternetService', 'PhoneService', 'PaymentMethod', 'TotalCharges', 'Contract',  'StreamingTV', 'OnlineBackup','Churn']]
# churn_df['churn'] = churn_df['churn'].astype('int')
churn_df.head()

# EDA

In [ ]:
import pandas_profiling

In [ ]:
profile = pandas_profiling.ProfileReport(churn_df)
profile

<h2>Drop Duplicates</h2>

In [ ]:
print(f'Shape before dropping duplicates {churn_df.shape}')
churn_df = churn_df.drop_duplicates()
print(f'Shape after dropping duplicates {churn_df.shape}')

In [ ]:
churn_df.head()

# Replace Categorical Values

In [ ]:
categorical_values_mapping = {"No":0,"Yes":1,"No phone service":-1,"No internet service":-1,
                              "Fiber optic":1,"DSL":2,
                              "Electronic check":0,"Mailed check":1,"Bank transfer (automatic)":2,"Credit card (automatic)":3,
                             "Month-to-month":0,"Two year":1,"One year":2
                             }
churn_df['Dependents'] = churn_df['Dependents'].map(categorical_values_mapping)
churn_df['MultipleLines'] = churn_df['MultipleLines'].map(categorical_values_mapping)
churn_df['InternetService'] = churn_df['InternetService'].map(categorical_values_mapping)
churn_df['PhoneService'] = churn_df['PhoneService'].map(categorical_values_mapping)
churn_df['PaymentMethod'] = churn_df['PaymentMethod'].map(categorical_values_mapping)
churn_df['Contract'] = churn_df['Contract'].map(categorical_values_mapping)
churn_df['StreamingTV'] = churn_df['StreamingTV'].map(categorical_values_mapping)
churn_df['OnlineBackup'] = churn_df['OnlineBackup'].map(categorical_values_mapping)
churn_df['Churn'] = churn_df['Churn'].map(categorical_values_mapping)

churn_df.head()

In [ ]:
# X = np.asarray(churn_df)
# X[0:5]

**Convert String to Float**

In [ ]:
# churn_df['TotalCharges'] = churn_df['TotalCharges'].fillna(0)
churn_df['TotalCharges'] = churn_df['TotalCharges'].replace({" ":'0'})
churn_df['TotalCharges'] = churn_df['TotalCharges'].astype(float)

Lets define X, and y for our dataset:

In [ ]:
X = np.asarray(churn_df[['tenure','Dependents', 'MultipleLines', 'InternetService', 'PhoneService', 'PaymentMethod', 'TotalCharges', 'Contract',  'StreamingTV', 'OnlineBackup']])
X[0:5]

In [ ]:
y = np.asarray(churn_df['Churn'])
y [0:5]

Normalize Data

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

## Train/Test dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.15,random_state=4)
print(f'Train set:{X_train.shape}, {y_train.shape}')
print(f'Test set:{X_test.shape}, {y_test.shape}')

<h2 id="modeling">Modeling (XGBClassifier from XGBoost)</h2>


XGBoost provides a wrapper class to allow models to be treated like classifiers or regressors in the scikit-learn framework.

This means we can use the full scikit-learn library with XGBoost models.

The XGBoost model for classification is called XGBClassifier. We can create and and fit it to our training dataset. Models are fit using the scikit-learn API and the model.fit() function.

Parameters for training the model can be passed to the model in the constructor. Here, we use the sensible defaults.
Now lets fit our model with train set:


In [ ]:
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier

LR = XGBClassifier().fit(X_train,y_train)
LR

In [ ]:
yhat = LR.predict(X_test)
yhat.shape

**predict_proba**  returns estimates for all classes, ordered by the label of classes. So, the first column is the probability of class 1, P(Y=1|X), and second column is probability of class 0, P(Y=0|X):


In [ ]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

<h2 id="evaluation">Evaluation</h2>


### jaccard index

Lets try jaccard index for accuracy evaluation. we can define jaccard as the size of the intersection divided by the size of the union of two label sets. If the entire set of predicted labels for a sample strictly match with the true set of labels, then the subset accuracy is 1.0; otherwise it is 0.0.


In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat)

### confusion matrix

Another way of looking at accuracy of classifier is to look at **confusion matrix**.


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test, yhat, labels=[1,0]))

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['churn=1','churn=0'],normalize= False,  title='Confusion matrix')

Look at first row. The first row is for customers whose actual churn value in test set is 1.
As you can calculate, out of 40 customers, the churn value of 15 of them is 1. 
And out of these 15, the classifier correctly predicted 6 of them as 1, and 9 of them as 0. 

It means, for 6 customers, the actual churn value were 1 in test set, and classifier also correctly predicted those as 1. However, while the actual label of 9 customers were 1, the classifier predicted those as 0, which is not very good. We can consider it as error of the model for first row.

What about the customers with churn value 0? Lets look at the second row.
It looks like  there were 25 customers whom their churn value were 0. 

The classifier correctly predicted 24 of them as 0, and one of them wrongly as 1. So, it has done a good job in predicting the customers with churn value 0. A good thing about confusion matrix is that shows the model’s ability to correctly predict or separate the classes.  In specific case of binary classifier, such as this example,  we can interpret these numbers as the count of true positives, false positives, true negatives, and false negatives. 


In [ ]:
print (classification_report(y_test, yhat))

Based on the count of each section, we can calculate precision and recall of each label:

-   **Precision** is a measure of the accuracy provided that a class label has been predicted. It is defined by: precision = TP / (TP + FP)

-   **Recall** is true positive rate. It is defined as: Recall =  TP / (TP + FN)

So, we can calculate precision and recall of each class.

**F1 score:**
Now we are in the position to calculate the F1 scores for each label based on the precision and recall of that label. 

The F1 score is the harmonic average of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0. It is a good way to show that a classifer has a good value for both recall and precision.

And finally, we can tell the average accuracy for this classifier is the average of the F1-score for both labels, which is 0.77 in our case.

From this we can see that XGBoost has improved our coutcome by 5% as compared to logistic regression.
<a href="https://www.kaggle.com/anmol4210/customerchurnpredictionusinglogisticregression"> Logistic Regression Notebook</a>

### log loss

Now, lets try **log loss** for evaluation. In XGBoost, the output can be the probability of customer churn is yes (or equals to 1). This probability is a value between 0 and 1.
Log loss( Logarithmic loss) measures the performance of a classifier where the predicted output is a probability value between 0 and 1. 


In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)